In [1]:
import pandas as pd
import utils
from plotly.io import show

from skfolio import Population, RiskMeasure, Portfolio
from skfolio.model_selection import cross_val_predict, CombinatorialPurgedCV
from skfolio.optimization import (
    MeanRisk,
    ObjectiveFunction,
    EqualWeighted,
    InverseVolatility,
)

# Fetch Data


In [2]:
tickers = [
    "TANLA",
    "FINEORG",
    "HINDOILEXP",
    "ASHOKA",
    "CAMS",
    "CUMMINSIND",
    "TEGA",
    "HDFCAMC",
]
# tickers = [
#    "GPIL",
#    "CUMMINSIND",
#    "HCLTECH",
#    "SUNPHARMA",
#    "TATAPOWER",
#    "ADANIGREEN",
#    "HDFCAMC",
#    "LTTS",
#    "NMDC",
#    "TEGA",
# ]

In [3]:
ret = utils.get_multiple_returns(tickers)
ret

,TANLA,FINEORG,HINDOILEXP,ASHOKA,CAMS,CUMMINSIND,TEGA,HDFCAMC
Date,,,,,,,,
2021-12-14,0.049977,0.018264,0.002723,-0.005184,0.007939,-0.030285,-0.091729,-0.010621
2021-12-15,0.007548,0.057425,-0.024165,0.004169,0.005709,-0.027288,-0.065130,-0.012243
2021-12-16,-0.032670,-0.008081,-0.008347,-0.014530,-0.026298,0.001333,0.022384,-0.007734
2021-12-17,-0.049689,-0.010081,-0.036756,-0.017904,-0.022359,-0.024554,-0.025067,-0.016327
2021-12-20,-0.022797,-0.056108,-0.028255,-0.045576,-0.012266,-0.023316,-0.074125,-0.035176
...,...,...,...,...,...,...,...,...
2024-02-13,0.026490,-0.005759,-0.017539,0.050826,-0.010903,-0.009523,-0.015111,0.035352
2024-02-14,-0.007819,-0.000385,0.079895,0.023442,0.005114,0.007864,0.052685,0.003573
2024-02-15,0.034147,-0.001087,0.114905,0.022035,0.024400,0.027408,0.059203,0.001257


In [4]:
nse = utils.get_returns("^NSEI", index=True)
nse.index = nse["Date"]
nse = nse.drop(columns=["Date"]).loc[ret.index[0] :]
nse

,^NSEI
Date,
2021-12-14,-0.002496
2021-12-15,-0.005974
2021-12-16,0.001568
2021-12-17,-0.015259
2021-12-20,-0.021843
...,...
2024-02-14,0.004452
2024-02-15,0.003237
2024-02-16,0.005931


# Models


### Sharpe Ratio Maximization


In [5]:
sharpe_model = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    portfolio_params=dict(name="Max Sharpe"),
)
sharpe_pred = sharpe_model.fit_predict(ret)
sharpe_pred.annualized_sharpe_ratio

2.046890202808289

### Minimum Variance


In [6]:
inv_var_model = InverseVolatility(portfolio_params=dict(name="Minimum Variance"))
inv_var_pred = inv_var_model.fit_predict(ret)
inv_var_pred.annualized_sharpe_ratio

1.2546097575506192

### Equally Weighted


In [7]:
equ_model = EqualWeighted(portfolio_params=dict(name="Equal Weight"))
equ_pred = equ_model.fit_predict(ret)
equ_pred.annualized_sharpe_ratio

1.0674837753460373

### NIFTY50 Benchmark


In [8]:
benchmark = EqualWeighted(portfolio_params=dict(name="NIFTY50"))
bench_pred = benchmark.fit_predict(nse)
bench_pred.annualized_sharpe_ratio

0.8732821419217808

### Clustering Optimization


In [9]:
import skfolio.optimization as opt

inner_estimator = MeanRisk(
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    risk_measure=RiskMeasure.VARIANCE,
)
outer_estimator = opt.RiskBudgeting(risk_measure=RiskMeasure.CVAR)

cluster_model = opt.NestedClustersOptimization(
    inner_estimator=inner_estimator,
    outer_estimator=outer_estimator,
    n_jobs=-1,
    portfolio_params=dict(name="NCO-1"),
)
cluster_pred = cluster_model.fit_predict(ret)
cluster_pred.annualized_sharpe_ratio

1.131608205892342

### Rolling Window Optimization


In [10]:
rolling_pred = utils.rolling_window_portfolio(ret, 5)

In [11]:
population = Population(
    [cluster_pred, sharpe_pred, inv_var_pred, equ_pred, rolling_pred, bench_pred]
)
population.set_portfolio_params(compounded=True)
population.plot_composition()

In [12]:
population.plot_cumulative_returns()

In [13]:
summary = population.summary()
summary[summary.index.str.contains("Annualized")]

,NCO-1,Max Sharpe,Minimum Variance,Equal Weight,Rolling Window,NIFTY50
Annualized Mean,22.25%,43.92%,24.10%,21.93%,12.14%,12.27%
Annualized Variance,3.87%,4.61%,3.69%,4.22%,7.13%,1.97%
Annualized Semi-Variance,1.97%,2.28%,1.97%,2.24%,3.44%,1.07%
Annualized Standard Deviation,19.66%,21.46%,19.21%,20.54%,26.70%,14.05%
Annualized Semi-Deviation,14.04%,15.10%,14.02%,14.98%,18.55%,10.34%
Annualized Sharpe Ratio,1.13,2.05,1.25,1.07,0.45,0.87
Annualized Sortino Ratio,1.59,2.91,1.72,1.46,0.65,1.19


In [14]:
population.composition() * 1_00_000

,NCO-1,Max Sharpe,Minimum Variance,Equal Weight,Rolling Window,NIFTY50
asset,,,,,,
CAMS,19941.787340,0.000000,15672.850681,12500.0,5600.176560,0.0
CUMMINSIND,19059.850677,68098.175260,17026.218059,12500.0,3035.047242,0.0
FINEORG,18016.416692,6654.013799,12197.559545,12500.0,29915.091907,0.0
TEGA,16397.246256,11040.759258,11710.312731,12500.0,2509.284246,0.0
HINDOILEXP,12128.256091,0.000000,9011.019839,12500.0,15368.117235,0.0
TANLA,9134.558767,0.000000,8540.531059,12500.0,27268.977992,0.0
HDFCAMC,2676.045187,7085.761330,15085.512055,12500.0,1857.380791,0.0
ASHOKA,2645.838991,7121.288175,10755.996032,12500.0,14445.923982,0.0
^NSEI,0.000000,0.000000,0.000000,0.0,0.000000,100000.0


# Cross Validation


In [15]:
from sklearn.model_selection import KFold


pred = cross_val_predict(cluster_model, ret, cv=KFold())
pred.annualized_sharpe_ratio

SolverError: Solver 'CLARABEL' failed for . Try another solver, or solve with solver_params=dict(verbose=True) for more information